In [3]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2018, day=17)

def parses(data):
    return [parse.parse('{}={:d}, {}={:d}..{:d}', line).fixed 
            for line in data.strip().split('\n')]

data = parses(puzzle.input_data)

In [4]:
sample = parses("""x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504""")

In [5]:
def render(scan,x,y,maxy=None):
    scan = scan.copy()
    scan[x,y] = 4
    minx = min(x for x,y in scan)
    maxx = max(x for x,y in scan)
    miny = min(y for x,y in scan)
    if maxy is None:
        maxy = max(y for x,y in scan)
    s = ""
    for y in range(miny,maxy+1):
        for x in range(minx,maxx+1):
            s += ' #~|X'[scan[x,y]]
        s += '\n'
    print(s)

In [41]:
def solve_a(data):
    scan = defaultdict(int)
    SAND, CLAY, WATER, RUNNING = 0, 1, 2, 3
    
    for a, n, b, m1, m2 in data:
        for m in range(m1,m2+1):
            if a == 'x':
                scan[n,m] = CLAY
            else:
                scan[m,n] = CLAY

    stack = [(500,0)]
    ymax = max(y for x,y in scan)
    ymin = min(y for x,y in scan)
    history = []

    while stack:
        x, y = stack.pop()
        scan[x,y] = RUNNING
        
        if y >= ymax or scan[x,y+1] == RUNNING:
            continue
        
        if scan[x,y+1] == SAND and y+1 <= ymax:
            stack.append((x,y+1))
            continue
        
        down = []
        if scan[x,y+1] in [CLAY, WATER]:
            new = []
            still = True
            for side in (1,-1):
                for x2 in itertools.count(x+side,side):
                    if scan[x2,y] in [CLAY, WATER]:
                        break
                    
                    if scan[x2,y+1] == RUNNING:
                        still = False
                        break
                    
                    scan[x2,y] = RUNNING
                    new.append((x2,y))
                    if scan[x2,y+1] == SAND and y+1 <= ymax:
                        down.append((x2,y+1))
                        still = False
                        break
            if still:
                for x2, y in new:
                    scan[x2,y] = WATER
            
        
        if still:
            scan[x,y] = WATER
            if scan[x,y-1] == RUNNING:
                stack.append((x,y-1))
        if down:
            stack.append((x,y))
            stack.extend(down)
#     render(scan,x,y) #,maxy=50)
#     print('-----------------------------------------'*2, '-----------------------------------------'*2)
    return sum(1 for x,y in scan if scan[x,y] in [WATER, RUNNING] and ymin <= y <= ymax) 
#     return scan

In [48]:
def fill(data):
    # Fully iterative solution
    scan = defaultdict(int)
    SAND, CLAY, WATER, RUNNING = 0, 1, 2, 3
    
    # Parse into map dictionary
    for a, n, b, m1, m2 in data:
        for m in range(m1,m2+1):
            if a == 'x':
                scan[n,m] = CLAY
            else:
                scan[m,n] = CLAY
    
    # Start at prespecified source
    stack = [(500,0)]
    ymax, ymin = max(y for x,y in scan), min(y for x,y in scan)

    while stack:
        x, y = stack.pop()
        scan[x,y] = RUNNING
        below = (x,y+1)
        
        # Out of bounds or already explored
        if y >= ymax or scan[below] == RUNNING:
            continue
        
        # Continue downwards
        if scan[below] == SAND:
            stack.append(below)
            continue
        
        children = []
        filled = [(x,y)]
        still = True # assume we'll hit a wall
        for side in (1, -1):
            for x2 in itertools.count(x+side, side):
                if scan[x2,y] == CLAY:
                    break # Hit a wall
                scan[x2,y] = RUNNING
                filled.append((x2,y))
    
                below2 = (x2,y+1)
                if scan[below2] == SAND:
                    children.append(below2)
                
                if scan[below2] in [SAND, RUNNING]:
                    still = False
                    break
        
        if still:
            for x2, y in filled:
                scan[x2,y] = WATER
            above = (x,y-1)
            if scan[above] == RUNNING:
                stack.append(above)
        if children:
            # Keep in stack, but priotize children
            stack.append((x,y))
            stack.extend(children)
    sum_still = sum(1 for x,y in scan if scan[x,y] == WATER and ymin <= y <= ymax)
    sum_running = sum(1 for x,y in scan if scan[x,y] == RUNNING and ymin <= y <= ymax)
    return sum_still, sum_running

def solve_a(data):
    return sum(fill(data))

def solve_b(data):
    return fill(data)[0]

In [49]:
solve_a(sample)

57

In [50]:
solve_a(data)

37649

In [51]:
solve_b(sample)

29

In [52]:
solve_b(data)

30112

In [ ]:
# render(*H[-1], 13)

In [ ]:
render(solve_a(data), 500, 0)

In [ ]:
render(*H[-1])

In [ ]:
# render(*H[-1])

In [16]:
_, scan = solve_a(data)

In [18]:
mysettled = set( pos for pos, val in scan.items() if val == 2)
myflowing = set( pos for pos, val in scan.items() if val == 3)

In [25]:
refsettled = set([pt for pt in settled if ymin <= pt[1] <= ymax])
refflowing = set([pt for pt in flowing if ymin <= pt[1] <= ymax])

In [27]:
mysettled - refsettled

set()

In [28]:
myflowing - refflowing

{(500, 0), (500, 1), (500, 2), (500, 3), (500, 4), (500, 5)}

In [21]:
len(mysettled) + len(myflowing)

37655

In [13]:
37649

37649

In [24]:
len(settled)+len(flowing)

67767

In [23]:
import collections

clay = collections.defaultdict(bool)

for line in puzzle.input_data.strip().split('\n'):
    a, brange = line.split(',')
    if a[0] == 'x':
        x = int(a.split('=')[1])
        y1, y2 = map(int, brange.split('=')[1].split('..'))

        for y in range(y1, y2 + 1):
            clay[(x, y)] = True
    else:
        y = int(a.split('=')[1])
        x1, x2 = map(int, brange.split('=')[1].split('..'))

        for x in range(x1, x2 + 1):
            clay[(x, y)] = True

ymin, ymax = min(clay, key=lambda p: p[1])[1], max(clay, key=lambda p: p[1])[1]

settled = set()
flowing = set()

def fill(pt, direction=(0, 1)):
    flowing.add(pt)

    below = (pt[0], pt[1] + 1)

    if not clay[below] and below not in flowing and 1 <= below[1] <= ymax:
        fill(below)

    if not clay[below] and below not in settled:
        return False

    left = (pt[0] - 1, pt[1])
    right = (pt[0] + 1, pt[1])

    left_filled = clay[left] or left not in flowing and fill(left, direction=(-1, 0))
    right_filled = clay[right] or right not in flowing and fill(right, direction=(1, 0))

    if direction == (0, 1) and left_filled and right_filled:
        settled.add(pt)

        while left in flowing:
            settled.add(left)
            left = (left[0] - 1, left[1])

        while right in flowing:
            settled.add(right)
            right = (right[0] + 1, right[1])

    return direction == (-1, 0) and (left_filled or clay[left]) or \
        direction == (1, 0) and (right_filled or clay[right])

fill((500, 0))

print('part 1:', len([pt for pt in flowing | settled if ymin <= pt[1] <= ymax]))
# print('part 2:', len([pt for pt in settled if ymin <= pt[1] <= ymax]))

part 1: 37649
